In [ ]:
import numpy as np
import seqpro as sp

In [20]:
# Grab each available alphabet
dna = sp.alphabets.DNA
rna = sp.alphabets.RNA
aa = sp.alphabets.AA

In [ ]:
# Generate

In [23]:
# Generate some random sequences
test_seqs = sp.random_seqs((10, 15), alphabet=dna)

In [34]:
# Generate some random sequences 
test_seqs = sp.random_seqs(10, 15, alphabet=["A", "g", "C", "T"])
test_seq = test_seqs[0]

# Add seqs with only Ns
test_seqs_with_only_N = sp.random_seqs(10, 15, alphabet=["N"])

# Add seqs with Ns 
test_seqs_with_N = sp.random_seqs(10, 15, alphabet=["A", "G", "C", "T", "N"])

# Add some seqs of a different length
test_seqs_different_length = sp.random_seqs(10, 20, alphabet=["A", "G", "C", "T"])

# Concatenate all the seqs
all_test_seqs = np.concatenate([test_seqs, test_seqs_with_N, test_seqs_with_only_N, test_seqs_different_length])

# Check the lengths
len(test_seqs), len(test_seqs_with_N), len(test_seqs_with_only_N), len(all_test_seqs)

(10, 10, 10, 40)

In [35]:
def test_remove_only_N_seqs(seqs):
    seqs_with_no_only_N = sp.remove_only_N_seqs(seqs)
    assert len(seqs_with_no_only_N) == 30
    return seqs_with_no_only_N
test_seqs_with_no_only_N = test_remove_only_N_seqs(all_test_seqs)

In [ ]:
def test_remove_N_seqs(seqs):
    seqs_with_no_N = sp.remove_N_seqs(seqs)
    assert len(seqs_with_no_N) == 20
    return seqs_with_no_N
test_seqs_with_no_N = test_remove_N_seqs(test_seqs_with_no_only_N)

In [18]:
def test_sanitize_seqs(seqs):
    seqs = sp.sanitize_seqs(seqs)
    assert all([all([x in ["A", "G", "C", "T"] for x in seq]) for seq in seqs])
    return seqs
test_seqs_sanitized = test_sanitize_seqs(test_seqs_with_no_N)

In [19]:
def test_pad_seqs(seqs, pad="left", pad_value="N"):
    seqs = sp.pad_seqs(seqs, pad=pad, pad_value=pad_value)
    assert all([len(seq) == 20 for seq in seqs])
    return seqs
test_seqs_left_padded = test_pad_seqs(test_seqs_sanitized, pad="left")
test_seqs_right_padded = test_pad_seqs(test_seqs_sanitized, pad="right")
test_seqs_center_padded = test_pad_seqs(test_seqs_sanitized, pad="both")

In [20]:
def test_ohe(seqs, alphabet=sp.ALPHABETS["DNA"]):
    ohe_seqs = sp.ohe(seqs, alphabet=alphabet)
    assert ohe_seqs.shape == (20, 20, 4)
    return ohe_seqs
test_ohe_seqs = test_ohe(test_seqs_left_padded)

In [21]:
def test_length(seqs):
    lengths = sp.length(seqs)
    assert all([x == 20 for x in lengths])
    return lengths
test_lengths = test_length(test_seqs_left_padded)

In [22]:
def test_gc_content(seqs, normalize=True, length_axis=None, alphabet=None, ohe_axis=None):
    gc = sp.gc_content(seqs, normalize=normalize)
    if normalize:
        assert all([x >= 0 and x <= 1 for x in gc])
    else:
        assert all([x >= 0 and x <= 20 for x in gc])
    return gc
#test_gc_normalized = test_gc_content(test_seqs_left_padded, normalize=True)
#test_gc = test_gc_content(test_seqs_sanitized, normalize=False)
#test_gc_ohe = test_gc_content(test_ohe_seqs, normalize=False, alphabet=sp.ALPHABETS["DNA"], ohe_axis=1)

In [23]:
def test_nucleotide_content(seqs, normalize=True, length_axis=None, alphabet=None):
    nuc_content = sp.nucleotide_content(seqs, normalize=normalize)
    if normalize:
        assert all([x >= 0 and x <= 1 for x in nuc_content])
    else:
        assert all([x >= 0 and x <= 20 for x in nuc_content])
    return nuc_content
#test_nuc_content_normalized = test_nucleotide_content(test_seqs_sanitized, normalize=True)
#test_nuc_content = test_nucleotide_content(test_seqs_sanitized, normalize=False)
#test_nuc_content_ohe = test_nucleotide_content(test_ohe_seqs, normalize=False, alphabet=sp.ALPHABETS["DNA"], length_axis=1)

In [24]:
def test_count_kmers_seq(seq, k=3):
    kmers = sp._analyzers.count_kmers_seq(seq, k=k)
    assert len(kmers) == 4**k
    return kmers
#test_1mers = test_count_kmers_seq(test_seq, k=1)
#test_2mers = test_count_kmers_seq(test_seq, k=2) 
#test_3mers = test_count_kmers_seq(test_seq, k=3)

In [25]:
# import ordered dict
from collections import OrderedDict

def naive_count_kmers(seq, k):
    # Naive implementation of counting kmers

    # Initialize the dictionary
    kmers = OrderedDict()
    
    # Loop over the sequence
    for i in range(len(seq) - k + 1):
        kmer = seq[i:i+k]
        if kmer in kmers:
            kmers[kmer] += 1
        else:
            kmers[kmer] = 1
    
    # Return the dictionary sorted by the keys
    return OrderedDict(sorted(kmers.items()))

In [26]:
def test_k_shuffle(seqs, k=1, length_axis=1, seed=13):
    shuffled_seqs = sp.k_shuffle(seqs, k=k, length_axis=length_axis)
    return shuffled_seqs
test_1_shuffle_seqs = test_k_shuffle(test_seqs_left_padded, k=1, length_axis=1)
test_2_shuffle_seqs = test_k_shuffle(test_seqs_left_padded, k=2, length_axis=1)
test_3_shuffle_seqs = test_k_shuffle(test_seqs_left_padded, k=3, length_axis=1)

In [28]:
def test_k_shuffle_ohe(seqs, k=1, length_axis=1, alphabet=sp.ALPHABETS["DNA"]):
    shuffled_seqs = sp.k_shuffle(seqs, k=k, length_axis=length_axis, alphabet=alphabet)
    return shuffled_seqs
#test_1_shuffle_seqs_ohe = test_k_shuffle_ohe(test_ohe_seqs, k=1, length_axis=1, alphabet=sp.ALPHABETS["DNA"])
#test_2_shuffle_seqs_ohe = test_k_shuffle_ohe(test_ohe_seqs, k=2, length_axis=1, alphabet=sp.ALPHABETS["DNA"])
#test_3_shuffle_seqs_ohe = test_k_shuffle_ohe(test_ohe_seqs, k=3, length_axis=1, alphabet=sp.ALPHABETS["DNA"])

In [29]:
test_ohe_seqs_no_pad = test_ohe_seqs[10:]
test_1_shuffle_seqs_ohe_no_pad = test_k_shuffle_ohe(test_ohe_seqs_no_pad, k=1, length_axis=1, alphabet=sp.ALPHABETS["DNA"])
test_2_shuffle_seqs_ohe_no_pad = test_k_shuffle_ohe(test_ohe_seqs_no_pad, k=2, length_axis=1, alphabet=sp.ALPHABETS["DNA"])
test_3_shuffle_seqs_ohe_no_pad = test_k_shuffle_ohe(test_ohe_seqs_no_pad, k=3, length_axis=1, alphabet=sp.ALPHABETS["DNA"])

In [30]:
for ohe_seq in test_ohe_seqs_no_pad:
    decoded_seq_bytes = sp.ALPHABETS["DNA"].ohe_to_bytes(ohe_seq)
    decoded_seq = b"".join(decoded_seq_bytes).decode("ascii")
    original_kmers = naive_count_kmers(decoded_seq, k=2)
    print(original_kmers)

    shuffled_ohe_seq = sp.k_shuffle(ohe_seq, k=2, length_axis=0, alphabet=sp.ALPHABETS["DNA"])
    decoded_shuffled_seq_bytes = sp.ALPHABETS["DNA"].ohe_to_bytes(shuffled_ohe_seq)
    decoded_shuffled_seq = b"".join(decoded_shuffled_seq_bytes).decode("ascii")
    shuffled_kmers = naive_count_kmers(decoded_shuffled_seq, k=2)
    print(shuffled_kmers)

    print(f"Original kmers == Shuffled kmers: {original_kmers == shuffled_kmers}")
    print()

OrderedDict([('AA', 3), ('AC', 2), ('AG', 1), ('CA', 1), ('CG', 1), ('CT', 1), ('GA', 1), ('GG', 3), ('GT', 2), ('TA', 2), ('TC', 1), ('TT', 1)])
OrderedDict([('AA', 3), ('AC', 2), ('AG', 1), ('CA', 1), ('CG', 1), ('CT', 1), ('GA', 1), ('GG', 3), ('GT', 2), ('TA', 2), ('TC', 1), ('TT', 1)])
Original kmers == Shuffled kmers: True

OrderedDict([('AA', 2), ('AG', 2), ('CA', 1), ('CG', 1), ('GA', 2), ('GC', 1), ('GG', 1), ('GT', 2), ('TC', 1), ('TG', 2), ('TT', 4)])
OrderedDict([('AA', 2), ('AG', 2), ('CA', 1), ('CG', 1), ('GA', 2), ('GC', 1), ('GG', 1), ('GT', 2), ('TC', 1), ('TG', 2), ('TT', 4)])
Original kmers == Shuffled kmers: True

OrderedDict([('AA', 2), ('AG', 1), ('AT', 3), ('CT', 2), ('GA', 1), ('GC', 1), ('GG', 1), ('GT', 1), ('TA', 2), ('TC', 1), ('TG', 2), ('TT', 2)])
OrderedDict([('AA', 2), ('AG', 1), ('AT', 3), ('CT', 2), ('GA', 1), ('GC', 1), ('GG', 1), ('GT', 1), ('TA', 2), ('TC', 1), ('TG', 2), ('TT', 2)])
Original kmers == Shuffled kmers: True

OrderedDict([('AA', 1), ('

In [115]:
from seqpro._utils import _check_axes

In [ ]:
_check_axes(test_ohe_seqs_no_pad, length_axis=0, ohe_axis=False)

In [116]:
False == 0

True

In [ ]:
from seqpro import _deprecated as _dep

In [90]:
for seq in test_seqs_sanitized:
    print(f"Sequence: {seq}")
    original_kmers = naive_count_kmers(seq, k=2)
    print(original_kmers)

    shuffled_seq_bytes = sp.k_shuffle(seq, k=2, seed=42)
    shuffled_seq = b"".join(shuffled_seq_bytes).decode("ascii")
    shuffled_kmers = naive_count_kmers(shuffled_seq, k=2)
    print(f"Shuffled sequence: {shuffled_seq}")
    print(shuffled_kmers)

    dinuc_shuffle_seq = _dep.dinuc_shuffle_seq(seq, rng=np.random.RandomState(42))
    print(f"Dinucleotide shuffled sequence: {dinuc_shuffle_seq}")
    print(naive_count_kmers(dinuc_shuffle_seq, k=2))

    print(f"Original kmers == Shuffled kmers: {original_kmers == shuffled_kmers}")
    print(f"Original kmers == Dinucleotide shuffled kmers: {original_kmers == naive_count_kmers(dinuc_shuffle_seq, k=2)}")
    print()

Sequence: CACACCAGACTCACC
OrderedDict([('AC', 4), ('AG', 1), ('CA', 4), ('CC', 2), ('CT', 1), ('GA', 1), ('TC', 1)])
Shuffled sequence: CCAGACACACACCTC
OrderedDict([('AC', 4), ('AG', 1), ('CA', 4), ('CC', 2), ('CT', 1), ('GA', 1), ('TC', 1)])
Dinucleotide shuffled sequence: CACACACAGACTCCC
OrderedDict([('AC', 4), ('AG', 1), ('CA', 4), ('CC', 2), ('CT', 1), ('GA', 1), ('TC', 1)])
Original kmers == Shuffled kmers: True
Original kmers == Dinucleotide shuffled kmers: True

Sequence: CCGGCGTACATGTTC
OrderedDict([('AC', 1), ('AT', 1), ('CA', 1), ('CC', 1), ('CG', 2), ('GC', 1), ('GG', 1), ('GT', 2), ('TA', 1), ('TC', 1), ('TG', 1), ('TT', 1)])
Shuffled sequence: CGCGTTATGGTCCAC
OrderedDict([('AC', 1), ('AT', 1), ('CA', 1), ('CC', 1), ('CG', 2), ('GC', 1), ('GG', 1), ('GT', 2), ('TA', 1), ('TC', 1), ('TG', 1), ('TT', 1)])
Dinucleotide shuffled sequence: CCGCGTACATGGTTC
OrderedDict([('AC', 1), ('AT', 1), ('CA', 1), ('CC', 1), ('CG', 2), ('GC', 1), ('GG', 1), ('GT', 2), ('TA', 1), ('TC', 1), ('